# Impact scenario quality control

This notebook is intended to provide a review of the impact scenarios generated in the Severe Wind Hazard Assessment project. 

The impact results are generated using the [_HazImp_](https://github.com/GeoscienceAustralia/hazimp) tool, with TC wind footprints generated from [TCRM](https://github.com/GeoscienceAustralia/tcrm). Scenarios were selected in consultation with Department of Fire and Emergency Services (Western Australia) and the Bureau of Meteorology (Western Australia Regional Office).

### Dependencies

1. Numpy
2. Pandas
3. GeoPandas
4. Matplotlib
5. Seaborn
6. HazImp

NOTE: I have only successfully installed GeoPandas on a Python 3.6 installation (on a Windows environment). 

In [1]:
%matplotlib inline

import os 
from os.path import join as pjoin
import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

import seaborn as sns
sns.set_context('notebook', rc={"xtick.labelsize":"small", "ytick.labelsize":"small"})
sns.set_style('ticks')
palette = sns.blend_palette(["#5E6A71", "#006983", "#72C7E7", "#A33F1F",
                             "#CA7700", "#A5D867", "#6E7645"], 7)
dmgpal = sns.blend_palette([(0.000, 0.627, 0.235), (0.412, 0.627, 0.235), 
                            (0.663, 0.780, 0.282), (0.957, 0.812, 0.000), 
                            (0.925, 0.643, 0.016), (0.835, 0.314, 0.118),
                            (0.780, 0.086, 0.118)], 5)
sns.set_palette(palette)

The output files for the impact scenarios are stored in the project directory, and are stored as csv files. The vulnerability curves themselves are provided with the HazImp software, so we also import parts of the HazImp code.

In [2]:
from core_hazimp.jobs.vulnerability_model import vuln_sets_from_xml_file

The scenarios represent the impacts of category 3 and 5 cyclones on communities in WA's northern coastline. 

In [3]:
nrows = 9
ncols = 3
hcell, wcell = 1., 1.5
hpad, wpad = 0.2, 0.25
f, ax = plt.subplots(figsize=((nrows*hcell+hpad), (ncols*wcell+wpad)))
ax.axis('off')
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
for sp, att in ax.spines.items():
    att.set_color('w')
    att.set_zorder(0)
    
titlestr = "Event number | Location | Intensity "
columns = titlestr.split('|')
cell_text = [columns]
cell_colors = [['0.75']*len(columns)]
cell_text.append(['000-01322', 'Exmouth', 'Category 3'])
cell_text.append(['000-06481', 'Exmouth', 'Category 5'])
cell_text.append(['012-06287', 'Broome', 'Category 3'])
cell_text.append(['012-03435', 'Broome', 'Category 5'])
cell_text.append(['006-00850', 'Karratha', 'Category 3'])
cell_text.append(['009-07603', 'Karratha', 'Category 5'])
cell_text.append(['003-03693', 'Port Hedland', 'Category 3'])
cell_text.append(['000-08534', 'Port Hedland', 'Category 5'])
cell_colors.append(['w']*len(columns))
cell_colors.append(['w']*len(columns))
cell_colors.append(['w']*len(columns))
cell_colors.append(['w']*len(columns))
cell_colors.append(['w']*len(columns))
cell_colors.append(['w']*len(columns))
cell_colors.append(['w']*len(columns))
cell_colors.append(['w']*len(columns))
the_table = plt.table(cellText=cell_text, loc='center',
                      fontsize=12, cellColours=cell_colors)
the_table.set_zorder(10)
the_table.auto_set_font_size(False)
the_table.set_fontsize(12)
the_table.scale(1, 2)
ax.set_title("Tropical cyclone scenarios for Severe Wind Hazard Assessment", fontsize=12)
plt.show()

To review an event, change the event number in the following cell. This will load the different output file for inspection.

In [4]:
event_num = 'bsh301999'
data_path = "X:/georisk/HaRIA_B_Wind/projects/dfes_swha/data/processed/HazImp/20181008/"
#data_path = "C:/WorkSpace/DFES/data/processed/HazImp/20181008/"
output_path = pjoin(data_path, event_num)
try:
    os.makedirs(output_path)
except:
    pass
event_file = pjoin(data_path, "DFES_{0}.csv".format(event_num))
YEAR_ORDER = ['1891 - 1913', '1914 - 1946', '1947 - 1961', '1962 - 1981', '1982 - 1996', '1997 - present']
ROOF_TYPE_ORDER = ['Metal Sheeting', 'Tiles', 'Fibro / asbestos cement sheeting']
WALL_TYPE_ORDER = ['Double Brick', 'Fibro / asbestos cement sheeting', 'Timber', 'Brick Veneer']

In [5]:
df = pd.read_csv(event_file)
#df = df[df['LGA_NAME']=='Exmouth (S)']
df

Start with plotting the structural loss ratio against 10m wind speed. This should intuitively follow the vulnerability functions used in _HazImp_. Points are coloured by the construction age. 

In [7]:
g = sns.lmplot(x='0.2s gust at 10m height m/s',
               y='structural_loss_ratio', 
               hue='YEAR_BUILT',
               hue_order=YEAR_ORDER,
               data=df,
               fit_reg=False,
               aspect=2, legend_out=False)

g.ax.set_ylabel("Structural loss ratio")
g.ax.set_ylim((0, 1))
g.ax.set_xlim((20, 90))
g.ax.legend(title="Construction era", frameon=True)
plt.savefig(pjoin(output_path, "SLR_Windspeed_by_age.pdf"), dpi=600)

Examine the distribution of structural loss ratio within each contruction era, and colour the points by roof type (out of interests sake). For lower intensity events, we expect the newer construction (post-1982) to experience very low or no damage, since these events will generate wind speeds that are well below the design wind speeds for the region.

In [8]:
fig, ax = plt.subplots()
sns.stripplot(x='YEAR_BUILT', y='structural_loss_ratio', hue='ROOF_TYPE', order=YEAR_ORDER, data=df, jitter=True,
              ax=ax)
ax.set_xlabel("Construction era")
ax.set_ylim((0, 1))

ax.set_ylabel("Structural loss ratio")
ax.legend(title="Roof material", frameon=True)
fig.savefig(pjoin(output_path, "SLR_by_age.pdf"), dpi=600, bbox_inches="tight")

Now look at the structural loss in dollar terms. This is less important for an EM perspective, but serves as a cursory check that the outputs are sensible. Replacement costs are based on quantity surveyor data and represent 2017 values.

In [9]:
g= sns.lmplot(x='0.2s gust at 10m height m/s',
              y='structural_loss', 
              hue='YEAR_BUILT', 
              hue_order=YEAR_ORDER,
              data=df,
              fit_reg=False,
              aspect=2, legend_out=False)

ylabels = g.ax.get_yticks()
g.ax.set_yticklabels(["${:0,.0f}".format(y) for y in ylabels])
g.ax.set_ylabel("Structural loss ($)")
g.ax.set_xlim((20, 90))
g.ax.legend(title="Construction era", frameon=True)
plt.savefig(pjoin(output_path, "SLC_Windspeed_by_age.pdf"), dpi=600, bbox_inches="tight")

In [10]:
" ${:,.2f}".format(df.structural_loss.sum())

In [11]:
ax = sns.distplot(df['structural_loss_ratio'], bins=np.arange(0, 1.01, 0.05), kde=False, axlabel="Structural loss ratio",
                  hist_kws={"linewidth":3, "width":0.04})

ax.set_ylabel("Count")
ax.set_xlim((0.0, 1.0))
ax.set_yscale('log')

In [12]:
bins=[0.0, 0.02, 0.1, 0.2, 0.5, 1.0]
labels=['Negligible', 'Slight', 'Moderate', 'Extensive', 'Complete']
df['Damage state'] = pd.cut(df['structural_loss_ratio'], bins, right=False, labels=labels)

df[df['Damage state'].isin(['Moderate', 'Extensive','Complete'])]['SA2_NAME'].value_counts()

In [13]:
damage_states = df.groupby(['Damage state', 'LGA_NAME']).count()['LID']

In [12]:
damage_building_type = df.groupby(['Damage state', 'YEAR_BUILT', 'WALL_TYPE', 'ROOF_TYPE']).count()['LID']

In [13]:
damage_building_type.to_excel(pjoin(output_path, "{0}_dmgstate_building_type.xlsx".format(event_num)))

In [14]:
damage_states.to_excel(pjoin(output_path, "{0}_dmgstate.xlsx".format(event_num)))

In [13]:
sns.countplot(x='Damage state', data=df, order=labels, palette=dmgpal)
plt.savefig(pjoin(output_path, "damage_state.pdf"), dpi=600, bbox_inches="tight")

In [14]:
ax = sns.countplot(x='Damage state', data=df, order=labels, hue='YEAR_BUILT', hue_order=YEAR_ORDER)
ax.legend(title="Construction era", frameon=True, loc=1)
plt.savefig(pjoin(output_path, "damage_state_by_age.pdf"), dpi=600, bbox_inches="tight")

In [20]:
df.pivot_table(index='YEAR_BUILT', columns='Damage state', aggfunc='size', fill_value=0)

In [16]:
damage_state_lga = df.groupby(['Damage state', 'LGA_NAME'])
df.pivot_table(index='LGA_NAME', columns='Damage state', aggfunc='size', fill_value=0)

In [19]:
damage_building_type = df.groupby(['Damage state', 'YEAR_BUILT', 'WALL_TYPE', 'ROOF_TYPE'])
df.pivot_table(index=['Damage state', 'YEAR_BUILT'], columns=['WALL_TYPE', 'ROOF_TYPE'], aggfunc='size', fill_value=0)

In [17]:
ax = sns.pointplot(x='Damage state', y='structural_loss_ratio', data=df, order=labels,
                   hue='YEAR_BUILT', hue_order=YEAR_ORDER)
ax.set_ylabel("Structural loss ratio")
ax.set_ylim((0, 1))
ax.legend(title="Construction era", frameon=True)
plt.savefig(pjoin(output_path, "SLR_by_damage_state.png"), bbox_inches="tight")

Building age profile for the community. 

In [18]:
fig, ax = plt.subplots()
sns.countplot(x='YEAR_BUILT', order=YEAR_ORDER, data=df, ax=ax)
ax.set_xlabel("Construction era")

Structural loss ratio again. This time, we've plotted the mean structural loss ratio for each constuction era. The vertical bar indicates the standard deviation range in the loss outcomes for each category. 

In [91]:
fig,ax = plt.subplots()
sns.barplot(x='YEAR_BUILT', y='structural_loss_ratio', order=YEAR_ORDER, data=df, ax=ax)
ax.set_ylabel("Mean structural loss ratio")
ax.set_xlabel("Construction era")
ax.set_ylim((0, 1))
plt.savefig(pjoin(output_path, "Mean_SLR_by_age.png"), bbox_inches="tight")

Breakdown of structural loss ratio, building age and roof type. 

In [92]:
g = sns.FacetGrid(df, row='YEAR_BUILT', hue='ROOF_TYPE', aspect=2)

g = g.map(sns.barplot, 'ROOF_TYPE', 'structural_loss_ratio', order=ROOF_TYPE_ORDER)

In [93]:
g = sns.FacetGrid(df, row='YEAR_BUILT', hue='WALL_TYPE', aspect=2)

g = g.map(sns.barplot, 'WALL_TYPE', 'structural_loss_ratio', order=WALL_TYPE_ORDER)